# DSA4262 Data Visualization Individual Assignment 2

Cheryl Lee (Li Jia Xuan) A0262272J

Link to Github repository: https://github.com/cherylololol/DSA4262

## 1. Motivation

Mental health signals are often expressed through subjective, nuanced language rather than explicit clinical markers. In digital environments such as social media platforms, individuals frequently describe stress through informal narratives, emotional vocabulary, and subtle shifts in tone. Translating these unstructured expressions into measurable computational features presents a central challenge in Artificial Intelligence (AI) for mental health.

The Dreaddit dataset provides annotated Reddit posts labelled as "Stress" or "Non-Stress". These posts contain informal language, indirect expressions, and domain specific discourse patterns. This makes stress detection both linguistically complex and context dependent.

The objective of this assignment is to build a binary classification model capable of identifying stress in Reddit text while emphasizing interpretability and sense making. Beyond predictive performance, this study investigates which lexical, syntactic, and social signals contribute to stress detection, how model errors manifest across communities, and what ethical considerations arise in potential real world deployment.

By integrating psychologically grounded feature engineering with text based modelling, this work aims to balance predictive accuracy with transparency and responsible application.

## 2. Data Cleaning and Preprocessing

Prior to modelling, we apply structured preprocessing to ensure data integrity, reproducibility, and deployment realism.
The following subsections detail text normalization, metadata removal, and feature organization decisions.


### 2.1 Importing of Libraries

In [ ]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

import shap

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

### 2.2 Data Loading

We use the predefined Dreaddit training and test splits to ensure fair evaluation and prevent data leakage. 

In [ ]:
train_raw = pd.read_csv("data/dreaddit-train.csv")

print(train_raw.shape)
train_raw.head()

In [ ]:
test_raw  = pd.read_csv("data/dreaddit-test.csv")

print(test_raw.shape)
test_raw.head()

### 2.3 Data Integrity Checks

No missing values or duplicate entries were detected in either training or test sets.

In [ ]:
print("Train missing:\n", train_raw.isna().sum().sum())
print("Test missing:\n", test_raw.isna().sum().sum())

In [ ]:
print("Train duplicates:", train_raw.duplicated().sum())
print("Test duplicates:", test_raw.duplicated().sum())


### 2.4 Minimal Text Cleaning

We apply light preprocessing by:
- Removing rows with missing text or labels
- Ensuring labels are binary integers
- Removing URLs
- Trimming whitespace

We deliberately avoid aggressive normalization (e.g., stopword removal or lemmatization), as stress detection is sensitive to function words and self referential cues that may be removed by heavy preprocessing.


In [ ]:
def clean_text(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"http\S+|www\S+", "", s)
    return s

def basic_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["text", "label"]).copy()
    df["label"] = df["label"].astype(int)
    df["text"] = df["text"].apply(clean_text)
    return df

train_df = basic_clean(train_raw)
test_df  = basic_clean(test_raw)


### 2.5 Removing Non Generalizable Columns

Here, we remove identifier and annotation metadata columns (e.g., post_id, id, confidence, sentence_range). This prevents information leakage and ensures the model relies only on features available in real world deployment setting.

In [ ]:
drop_cols = ["id", "post_id", "sentence_range", "confidence"]
train_df = train_df.drop(columns=[c for c in drop_cols if c in train_df.columns])
test_df  = test_df.drop(columns=[c for c in drop_cols if c in test_df.columns])

print("Train cleaned:", train_df.shape)
print("Test cleaned :", test_df.shape)

### 2.6 Feature Group Identification

Following the Dreaddit feature taxonomy, we group numeric features into:

* Lexical features (LIWC, DAL, sentiment)
* Syntactic features (readability indices)
* Social media features (engagement related metrics)

This grouping allows systematic comparison of feature subsets during modelling.


In [ ]:
lex_cols = [c for c in train_df.columns if c.startswith("lex_")]
syn_cols = [c for c in train_df.columns if c.startswith("syntax_")]
soc_cols = [c for c in train_df.columns if c.startswith("social_")]

print("Lexical features:", len(lex_cols))
print("Syntactic features:", len(syn_cols))
print("Social features:", len(soc_cols))


Lexical features dominates with 102 variables, compared to 2 syntactic and 4 social features. This imbalance motivates later dimensionality reduction to mitigate overfitting from high dimensional lexical signals.

## 3. Exploratory Data Analysis

We conduct exploratory analysis on the training data to understand patterns associated with stress.

This section aims to:

1. Examine dataset balance.
2. Explore stress prevalence across subreddits.
3. Investigate narrative length differences.
4. Identify lexical and linguistic signals associated with stress.
5. Motivate feature selection and modelling choices.


### 3.1 Distribution of Stress vs Non-Stress Posts

In [ ]:
label_counts = train_df["label"].value_counts().sort_index()
labels = ["Non-Stress", "Stress"]

plt.figure(figsize=(9,7))
bars = plt.bar(labels, label_counts.values, color=sns.color_palette("viridis", 2))

for i, v in enumerate(label_counts.values):
    plt.text(i, v + 10, f"{v}\n({v/len(train_df)*100:.1f}%)", ha='center')

plt.title("Distribution of Stress vs Non-Stress Posts in Training Data")
plt.ylim(0, max(label_counts.values) + 200)
plt.ylabel("Number of Posts", fontsize=12)
plt.xlabel("Label", fontsize=12)
plt.show()


The dataset is approximately balanced between stressed and non-stressed posts. This supports the use of F1 score as the primary evaluation metric, as neither class overwhelmingly dominates.

### 3.2 Stress Prevalence Across Subreddits

In [ ]:
stress_rate = (train_df.groupby("subreddit")["label"].mean() * 100).sort_values()

plt.figure(figsize=(12, 7))
cmap = plt.get_cmap('viridis')
bars = plt.barh(
    stress_rate.index,
    stress_rate.values,
    color=cmap(stress_rate.values / stress_rate.values.max())
)

for i, v in enumerate(stress_rate.values):
    plt.text(v + 1, i, f"{v:.1f}%", va='center', fontweight='bold', fontsize=10)


plt.title("Stress Prevalence by Subreddit")
plt.xlabel("Proportion of Posts Labeled as Stress", fontsize=12)
plt.ylabel("Subreddit", fontsize=12)
plt.xlim(0, 100) # Percentage scale
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()


Stress rate is computed as the mean of the binary stress label within each subreddit. The Viridis gradient encodes increasing prevalence, with lighter bars indicating higher proportions of stress labeled posts.

Stress prevalence varies substantially across subreddit communities. Mental health–focused subreddits exhibit higher baseline stress rates, while general forums show lower levels. This heterogeneity suggests that stress expression is domain dependent and that classification performance may differ across contexts. Consequently, this necessitates a model that can generalize across different emotional intensities instead of a 'one size fits all' threshold.

### 3.3 Word Count Distribution by Label

In [ ]:
label_map = {0: "Non-Stress", 1: "Stress"}
train_df["Condition"] = train_df["label"].map(label_map)

train_df['word_count'] = train_df['text'].apply(lambda x: len(x.split()))

plt.figure(figsize=(12, 7))
sns.histplot(data=train_df, x='word_count', hue='Condition', kde=True, bins=50, 
             palette=["#440154", "#21918c"])

stress_avg = train_df[train_df['Condition'] == 'Stress']['word_count'].mean()
non_stress_avg = train_df[train_df['Condition'] == 'Non-Stress']['word_count'].mean()

plt.axvline(stress_avg, color='#440154', linestyle='--', alpha=0.8, 
            label=f'Stress Avg: {stress_avg:.0f} words')
plt.axvline(non_stress_avg, color='#21918c', linestyle='--', alpha=0.8, 
            label=f'Non-Stress Avg: {non_stress_avg:.0f} words')

plt.title('Distribution of Word Count in Stress vs Non-Stress Posts')
plt.xlabel('Number of Words', fontsize=12)
plt.ylabel('Number of Posts', fontsize=12)
plt.legend()
plt.show()


Stress posts are marginally longer on average and show a slightly heavier right tail. However, the distributions overlap substantially, indicating that text length alone is not a strong discriminator. This suggests that lexical content rather than length drives predictive signal, supporting the use of TF-IDF representations in subsequent modelling.

### 3.4 Exploratory Linguistic Signals

We visualize selected representative linguistic markers across psychological dimensions to explore potential differences between stress and non-stress posts. These features were selected for interpretability and theoretical relevance, allowing us to qualitatively assess how stressed and non-stressed posts differ before formal modelling.


In [ ]:
name_mapping = {
    "lex_liwc_negemo": "Negative Emotion",
    "lex_liwc_posemo": "Positive Emotion",
    "lex_liwc_anx": "Anxiety Terms",
    "lex_liwc_Tone": "Emotional Tone",
    "lex_liwc_Clout": "Clout",
    "lex_liwc_i": "First Person (I)",
    "lex_liwc_you": "Second Person (You)",
    "lex_liwc_social": "Social Words",
    "lex_liwc_Authentic": "Authenticity",
    "lex_liwc_Analytic": "Analytical Thinking",
    "lex_liwc_feel": "Feeling Words",
    "lex_liwc_function": "Function Words",
    "lex_liwc_Dic": "Dictionary Coverage",
    "lex_liwc_Apostro": "Apostrophes",
    "lex_dal_min_pleasantness": "Min. Pleasantness",
    "lex_dal_avg_pleasantness": "Avg. Pleasantness",
    "lex_dal_avg_activation": "Avg. Activation",
    "syntax_fk_grade": "Flesch-Kincaid Grade",
    "syntax_ari": "Automated Readability Index",
    "sentiment": "Sentiment",
    "social_karma": "Social Karma",
    "social_num_comments": "Social Comment Count",
    "social_upvote_ratio": "Social Upvote Ratio",
    "label": "Stress Label (1=Stress)"
}

def map_name(name):
    return name_mapping.get(name, name)

In [ ]:
key_features = ["lex_liwc_negemo", "lex_liwc_i", "lex_liwc_Clout", "sentiment"]

fig, axes = plt.subplots(2,2, figsize=(12,8))

for ax, feature in zip(axes.flatten(), key_features):
    sns.boxplot(data=train_df, x="Condition", y=feature, ax=ax, palette="viridis", hue="Condition")
    ax.set_title(map_name(feature))
    ax.set_xlabel("Condition")
    ax.set_ylabel(map_name(feature))

plt.tight_layout()
plt.show()

From the boxplots, several clear patterns emerge:

* Negative emotion is higher in stressed posts.
* First person pronoun usage (I) increases under stress, suggesting greater self focus.
* Clout decreases in stressed posts, indicating that users in distress express themselves with less confidence or social certainty.
* Sentiment shifts more negatively. 

While distributions overlap, the median differences support the model’s reliance on emotional and self referential language features.


### 3.5 Correlation Heatmap of Features

In [ ]:
selected_cols = [
    "lex_liwc_negemo",
    "lex_liwc_posemo",
    "lex_liwc_anx",
    "lex_liwc_i",
    "lex_liwc_Clout",
    "lex_liwc_Tone",
    "syntax_fk_grade",
    "syntax_ari",
    "sentiment",
    "label",
]

corr = train_df[selected_cols].corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 7))
ax = sns.heatmap(
    corr,
    mask=mask,
    cmap="viridis",          
    vmin=-1, vmax=1,
    center=0,
    annot=True,
    fmt=".2f",            
    linewidths=0.5,
    linecolor="white",
    cbar_kws={"label": "Pearson correlation (r)"},
)

ax.set_xticklabels([map_name(c) for c in corr.columns], rotation=35, ha="right")
ax.set_yticklabels([map_name(c) for c in corr.index], rotation=0)

plt.title("Correlation Heatmap of Selected Psychological Markers")
plt.tight_layout()
plt.show()

Stress is positively correlated with negative emotion, anxiety related terms, and first person pronouns, indicating greater emotional negativity and self focus in stressed posts. Conversely, stress is negatively correlated with Tone, Clout, and overall sentiment, reflecting reduced positivity and linguistic confidence.

The heatmap also reveals multicollinearity among several predictors. Flesch–Kincaid Grade and ARI are highly correlated (r ≈ 0.97), suggesting redundancy, and multiple LIWC features exhibit moderate intercorrelation. Although readability features show weaker direct association with stress, they capture structural aspects of expression that complement lexical signals. Correlation based feature selection is therefore applied in the modelling phase to reduce redundancy and improve stability.

## 4. Feature Selection and Model Preparation

The Dreaddit dataset provides high dimensional lexical, syntactic, and social features in addition to raw text. Building on exploratory findings, including all available predictors risks introducing redundancy and noise, particularly given the dominance of lexical variables. Correlation based filtering is applied on the training subset to preserve deployment practicality.

### 4.1 Train–Validation Split

To maintain statistical validity and prevent information leakage, the training data is split into training and validation subsets prior to feature selection. All selection and tuning decisions are based exclusively on the training subset.

In [ ]:
text_col = "text"
target_col = "label"
numeric_cols = lex_cols + syn_cols + soc_cols

X = train_df.drop(columns=[target_col])
y = train_df[target_col]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("Total numeric features:", len(numeric_cols))
print("Training subset:", X_train.shape)
print("Validation subset:", X_val.shape)

### 4.2 Correlation Based Feature Selection

To reduce dimensionality and improve interpretability, Pearson correlation between each numeric feature and the stress label is computed on the training subset. Absolute correlation values are used to identify features with meaningful linear association.



In [ ]:
train_corr_df = X_train[numeric_cols].copy()
train_corr_df[target_col] = y_train.values

corr_to_label = train_corr_df.corr(numeric_only=True)[target_col].drop(target_col)

abs_corr = corr_to_label.abs().sort_values(ascending=False)
top_corr = abs_corr.sort_values(ascending=False).head(10)

colors = plt.cm.viridis(top_corr.values / top_corr.values.max())

fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(
    x=top_corr.values,
    y=[map_name(c) for c in top_corr.index],
    hue=[map_name(c) for c in top_corr.index], 
    palette="viridis_r",                      
    legend=False,
    ax=ax
)

for i, v in enumerate(top_corr.values):
    plt.text(v + 0.002, i, f"{v:.3f}", va='center')

ax.set_title("Top Numeric Features Correlated with Stress", fontsize=14, fontweight='bold')
ax.set_xlabel("Absolute Pearson Correlation coefficient", fontsize=12)
ax.set_ylabel("Feature", fontsize=12)
plt.tight_layout()
plt.show()

The figure shows the strongest predictors ranked by absolute Pearson correlation. Lexical emotional markers dominate the top associations, with Tone, Clout, and first person pronouns exhibiting the highest correlation with stress.

Multiple thresholds (|r| ≥ 0.1–0.4) were evaluated. 

In [ ]:
for t in [0.1, 0.2, 0.3, 0.4]:
    print(f"|r| ≥ {t}: {(abs_corr >= t).sum()} features selected")

A threshold of |r| ≥ 0.2 is chosen as it retained 15 features, balancing dimensionality reduction with signal preservation. It is important to note that Pearson correlation captures only linear relationships between predictors and the stress label. More complex non linear feature selection techniques could potentially identify additional signals. However, a correlation based filter was chosen to preserve interpretability, reduce dimensionality transparently, and align feature selection with theoretically grounded linguistic markers identified during exploratory analysis. 

In [ ]:
threshold = 0.2

selected_features = abs_corr[abs_corr >= threshold].index.tolist()

print("Selected features:", len(selected_features))
print(selected_features)

### 4.3 Multicollinearity Reduction

Pairwise correlations among the retained features were then examined to mitigate redundancy (|r| > 0.9). No feature pairs exceeded this threshold, indicating that the filtering step effectively produced a complementary and a non-collinear predictor set. Although readability indices were highly correlated in exploratory analysis, they did not surpass the label correlation threshold and were excluded prior to multicollinearity screening.

In [ ]:
selected_df = X_train[selected_features]

corr_matrix = selected_df.corr().abs()

upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

high_corr = [
    column for column in upper_triangle.columns
    if any(upper_triangle[column] > 0.9)
]

print("Highly correlated features to remove:", high_corr)

final_selected_features = [
    f for f in selected_features
    if f not in high_corr
]

print("Final feature count:", len(final_selected_features))

### 4.4 Final Feature Configurations for Modelling

We define three modelling configurations:

In [ ]:
text_only_features = None
text_plus_selected = final_selected_features
text_plus_all = numeric_cols

print("Text only model: uses TF-IDF from raw text only")
print("Text + Selected features:", len(text_plus_selected))
print("Text + All numeric features:", len(text_plus_all))

This structured comparison allows us to quantify the incremental contribution of engineered features beyond raw text.

## 5. Modelling Pipeline and Model Comparison

Following feature construction and selection in Section 4, we evaluate whether these representations enable reliable stress classification. 

A naive baseline is first established using a Dummy Classifier to determine minimum achievable performance without learning.
We then compare 3 common classifiers for text classification:

- **Logistic Regression**: linear model with interpretability.
- **Linear SVM**: strong performer for high dimensional sparse text.
- **Random Forest**: non-linear ensemble.

Models are trained on the training subset, selected based on validation F1 score, and evaluated on the test set.

### 5.1 Dummy Classifier

The baseline classifier predicts the majority class, providing a reference point to ensure our model captures genuine predictive signal.

In [ ]:
TEXT_COL = "text"

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train[TEXT_COL], y_train)

dummy_pred = dummy.predict(X_val[TEXT_COL])
baseline_f1 = f1_score(y_val, dummy_pred)

print(f"Baseline F1: {baseline_f1:.3f}")

### 5.2 Modelling Pipeline

To ensure consistency across models, a unified pipeline is constructed. Text features are transformed using TF-IDF, optionally combined with structured numeric features, and passed to the classifier.

In [ ]:
def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else X

def build_linear_pipeline(model, numeric_features=None, tfidf_max_features=5000):
    transformers = [
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2),
                                  min_df=5,
                                  max_features=tfidf_max_features),
         TEXT_COL)
    ]
    
    if numeric_features is not None and len(numeric_features) > 0:
        transformers.append(("num", StandardScaler(), numeric_features))
    
    preprocessor = ColumnTransformer(transformers, remainder="drop")
    
    pipe = Pipeline([
        ("preprocess", preprocessor),
        ("clf", model)
    ])
    return pipe

def build_rf_pipeline(numeric_features=None, tfidf_max_features=5000, random_state=42):
    transformers = [
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2),
                                  min_df=5,
                                  max_features=tfidf_max_features),
         TEXT_COL)
    ]
    if numeric_features is not None and len(numeric_features) > 0:
        transformers.append(("num", StandardScaler(), numeric_features))
    
    preprocessor = ColumnTransformer(transformers, remainder="drop")
    
    pipe = Pipeline([
        ("preprocess", preprocessor),
        ("to_dense", FunctionTransformer(to_dense, accept_sparse=True)),
        ("clf", RandomForestClassifier(
            n_estimators=400,
            max_depth=None,
            n_jobs=-1,
            random_state=random_state
        ))
    ])
    return pipe

def evaluate_model(pipe):
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_val)
    return f1_score(y_val, pred), pred

### 5.3 Model Comparison on Validation Set

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
    "Linear SVM": LinearSVC(max_iter=5000, random_state=42),
}

feature_sets = {
    "Text Only": [],
    "Text + Selected Numeric": final_selected_features,
    "Text + All Numeric": numeric_cols
}

results = []

for feat_name, num_feats in feature_sets.items():
    for model_name, model in models.items():
        pipe = build_linear_pipeline(model, num_feats, tfidf_max_features=5000)
        f1, _ = evaluate_model(pipe)
        results.append({"Model": model_name, "Features": feat_name, "Val F1": f1})
    
    rf_pipe = build_rf_pipeline(num_feats, tfidf_max_features=5000, random_state=42)
    f1, _ = evaluate_model(rf_pipe)
    results.append({"Model": "Random Forest", "Features": feat_name, "Val F1": f1})

results_df = pd.DataFrame(results).sort_values("Val F1", ascending=False).reset_index(drop=True)
results_df

Logistic Regression with Text + Selected Numeric features achieved the highest validation F1 score at ~0.792, outperforming Linear SVM and Random Forest across configurations.

Several patterns emerge:

1. Linear models outperform Random Forest: Stress classification in high dimensional TF-IDF space appears largely linearly separable. The non-linear capacity of Random Forest does not provide additional benefit and may introduce variance.

2. Selected numeric features outperform using all numeric features: Performance slightly drops when all engineered features are included (0.785 vs 0.792), suggesting that weaker or noisy predictors may dilute signal. Correlation-based filtering improves stability and reduces overfitting.

3. Feature augmentation improves over text only models: This indicates that structured psychological markers (e.g., negative emotion, self focus) provide incremental predictive value beyond lexical content alone.

Overall, Logistic Regression with selected engineered features provides the best balance between performance, interpretability, and model simplicity.

### 5.4 Cross Validation on Best Model

To evaluate robustness, five-fold stratified cross validation was performed on the best model: Logistic Regression with Text + Selected Numeric features.

In [ ]:
best_pipe = build_linear_pipeline(
    LogisticRegression(max_iter=2000, random_state=42),
    final_selected_features
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(best_pipe, X_train, y_train, cv=cv, scoring="f1")

print(f"5-fold CV F1 mean: {cv_scores.mean():.3f}")
print(f"5-fold CV F1 std : {cv_scores.std():.3f}")

Cross validation results demonstrate stable performance across folds, indicating that the selected feature configuration generalizes consistently within the training distribution. 

### 5.5 Hyperparameter Tuning

Logistic Regression includes a regularization parameter C, which controls the trade-off between model complexity and generalization. 

* Smaller C -> stronger regularization -> simpler decision boundary  
* Larger C -> weaker regularization -> more complex boundary  

In high-dimensional TF-IDF space, appropriate regularization is critical to prevent overfitting. We therefore perform grid search over multiple C values and select the model with the highest cross-validated F1-score.

In [ ]:
logreg_pipe = build_linear_pipeline(
    LogisticRegression(max_iter=2000, random_state=42),
    final_selected_features
)

param_grid = {
    "clf__C": [0.1, 0.5, 1, 2, 5, 10]
}

grid = GridSearchCV(
    logreg_pipe,
    param_grid=param_grid,
    scoring="f1",
    cv=cv,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

Grid search identified C = 2 as the optimal regularization strength, achieving a 5-fold cross validated F1 score of 0.785. 

Performance differences across C values were modest, suggesting that the model is relatively stable with respect to regularization strength. This indicates that predictive performance is driven primarily by feature representation rather than fine tuning of hyperparameters.

### 5.6 Final Evaluation on Test Set

In [ ]:
X_test = test_df.drop(columns=["label"])
y_test = test_df["label"]

final_model = grid.best_estimator_

test_pred = final_model.predict(X_test)

print("Test F1:", f1_score(y_test, test_pred))
print(classification_report(y_test, test_pred, target_names=["Non-Stress", "Stress"]))

The tuned Logistic Regression model achieves a test F1 score of 0.777 on the held out dataset, confirming strong generalization beyond validation data.

Recall for stress cases (0.81) exceeds precision (0.75), indicating that the model prioritizes identifying distressed individuals while accepting moderate false positives. In mental health screening contexts, this trade-off is often desirable, as failing to detect genuinely stressed individuals may carry greater risk than overflagging.

However, summary metrics do not reveal where errors occur or how performance varies across communities. These issues are examined in Section 6.

## 6. Analysis of Model Beyond Predictive Performance

While the overall test F1 score (≈ 0.777) indicates strong predictive performance, it does not explain how stress is expressed, where the model struggles, or what risks may arise in deployment.

To satisfy the “sense making” objective of this assignment, we move beyond aggregate metrics and analyse:

1. Model Behaviour
2. Performance variation across subreddits  
3. Failure patterns (false negatives and false positives)  
4. Structural factors such as post length  
5. Sensitivity to decision thresholds
6. Ethical considerations and deployment risks

Together, these analyses clarify both the strengths and limitations of the model in real world settings.

### 6.1 Model Behaviour

To interpret how the model makes predictions, SHAP (SHapley Additive exPlanations) values were computed on the test set. SHAP quantifies the contribution of each feature to the predicted probability of stress for individual samples by assigning each feature a contribution value for each post: values to the right increase the predicted probability of Stress, while values to the left push the prediction toward Non-Stress. Features are ranked by overall impact (mean absolute SHAP).

In [ ]:
final_model = grid.best_estimator_

test_pred = final_model.predict(X_test)
test_probs = final_model.predict_proba(X_test)[:, 1]

test_df_analysis = X_test.copy()
test_df_analysis["true_label"] = y_test.values
test_df_analysis["pred_label"] = test_pred
test_df_analysis["prob_stress"] = test_probs

print(test_df_analysis.head())

In [ ]:
def pretty_feature(name: str) -> tuple[str, str]:
    if name.startswith("num__"):
        raw = name.replace("num__", "")
        raw = raw.replace("lex_liwc_", "")
        raw = raw.replace("lex_dal_", "")
        raw = raw.replace("_", " ").strip()

        fixes = {
            "negemo": "Negative Emotion",
            "posemo": "Positive Emotion",
            "anx": "Anxiety",
            "i": "First-Person (I)",
            "clout": "Clout",
            "tone": "Emotional Tone",
            "apostro": "Apostrophes",
            "dic": "Dictionary words",
        }
        key = raw.lower().replace(" ", "")
        pretty = fixes.get(key, raw.title())

        return f"{pretty} (LIWC/DAL)", "LIWC/DAL"

    if name.startswith("tfidf__"):
        token = name.replace("tfidf__", "").strip()
        return f"{token} (TF-IDF)", "TF-IDF"

    return name, "Other"

In [ ]:
X_test_trans = final_model.named_steps["preprocess"].transform(X_test)
feature_names = final_model.named_steps["preprocess"].get_feature_names_out()

pretty_names = [pretty_feature(f)[0] for f in feature_names]

clf = final_model.named_steps["clf"]
explainer = shap.LinearExplainer(clf, X_test_trans, feature_names=pretty_names)
shap_values = explainer(X_test_trans)

In [ ]:
mean_shap = np.abs(shap_values.values).mean(axis=0)

top_idx = np.argsort(mean_shap)[-15:]

colors = cm.viridis(
    mean_shap[top_idx] / mean_shap[top_idx].max()
)

plt.figure(figsize=(12,8))
plt.barh(
    [feature_names[i] for i in top_idx],
    mean_shap[top_idx],
    color=colors
)

plt.gca()
plt.xlabel("Mean |SHAP Value|", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.title("Global Feature Importance", fontsize=14)
plt.tight_layout()
plt.show()

The global SHAP importance ranking indicates that engineered linguistic markers dominate model behaviour. Emotional tone exhibits the largest average contribution, followed by first person pronouns and negative emotion.

This confirms that stress classification is primarily driven by psychological expression patterns rather than isolated keywords. Structural markers such as clout and pleasantness also contribute meaningfully, suggesting that both affective polarity and linguistic confidence are central signals.

In contrast, individual TF-IDF tokens exhibit smaller average impact, indicating that lexical cues operate within broader emotional patterns rather than independently driving predictions.

In [ ]:
plt.figure(figsize=(10, 8))

shap.summary_plot(
    shap_values,
    X_test_trans,
    feature_names=feature_names,
    max_display=15,
    cmap="viridis", 
    show=False)

plt.title("Top Drivers of Stress Prediction")
plt.tight_layout()
plt.show()

While the barplot ranks feature importance globally, the beeswarm plot further reveals directional effects. The most influential predictors are engineered linguistic markers (LIWC/DAL). Features pushing predictions toward stress include:
* Lower emotional tone,
* Higher frequency of first person pronouns (“I”),
* Increased negative emotion terms,
* Lower linguistic clout (reduced confidence).

These patterns are consistent with Section 3.5 (correlation heatmap) and Section 4 (feature selection). The SHAP results therefore validate that the final model operationalises the same psychological signals identified during exploratory analysis.

Conversely, higher tone and greater linguistic confidence shift predictions toward non-stress, suggesting that emotionally positive and assertive language reduces the likelihood of stress classification.

Although engineered linguistic markers dominate the top positions, several TF-IDF tokens also appear among the most influential features. Many of these words occur within common constructions such as “I can’t even…” or “I don’t know what to do,” which convey overwhelm or perceived lack of control. These phrases act as contextual signals of distress and not as standalone keywords. Their presence reflects how the model captures specific linguistic patterns associated with stress expression.

Importantly, engineered linguistic features display broader and more consistent impact across samples than individual TF-IDF tokens. While tokens may spike importance for certain posts, psychological markers such as tone, clout, and negative affect contribute systematically across the dataset. This suggests that stress prediction is driven primarily by patterns of emotional expression and self-focus, rather than reliance on a small set of trigger words.

As the model is linear, coefficients provide global interpretability, while SHAP provides instance level explanations. The model is therefore transparent at both aggregate and individual levels.

### 6.2 Subreddit Level Performance

We evaluate subreddit level F1 to examine whether certain forms of stress are easier to detect.

In [ ]:
subreddit_stats = (
    test_df_analysis
    .groupby("subreddit")
    .apply(lambda x: pd.Series({
        "n": len(x),
        "F1": f1_score(x["true_label"], x["pred_label"])
    }))
    .sort_values("F1", ascending=False)
)

display(subreddit_stats)

Performance varies substantially across communities. Mental health–focused subreddits (e.g., ptsd, domesticviolence) show higher F1 scores. Whereas, relationship oriented or general discussion forums show lower performance, likely because posts contain mixed emotional signals (conflict, reconciliation, ambiguity) that blur the boundary between stress and non-stress. 

This suggests stress is expressed more explicitly in mental health communities and more indirectly in general domains. The variation indicates that stress is context dependent, and a single global model may not perform uniformly across communities. This domain dependence has implications for generalisation and fairness.

Performance estimates for subreddits with small sample sizes (e.g., n < 20) should be interpreted cautiously, as evaluation metrics may exhibit high variance under limited observations. Consequently, apparent performance differences in smaller communities may not generalise reliably.

### 6.3 Length Based Performance

To assess whether post length influences performance, F1 score was computed across length bins.

In [ ]:
test_df_analysis["word_count"] = test_df_analysis["text"].astype(str).apply(lambda x: len(x.split()))
test_df_analysis["length_bin"] = pd.qcut(test_df_analysis["word_count"], q=4)

length_perf = (
    test_df_analysis
    .groupby("length_bin")
    .apply(lambda x: f1_score(x["true_label"], x["pred_label"]))
)

length_perf

Performance remains relatively stable across most bins, with only minor variation. Both stressed and non-stressed posts exhibit overlapping length distributions, indicating that length alone is not a strong discriminative signal.

This suggests the model does not rely heavily on verbosity as a proxy for distress. Instead, classification appears driven primarily by linguistic content rather than structural length differences.

### 6.4 Threshold Sensitivity

By default, logistic regression uses a probability threshold of 0.5 to classify stress. However, in mental health applications, the cost of false negatives may outweigh false positives.

We therefore examine how precision and recall change under different thresholds.

In [ ]:
y_true = test_df_analysis["true_label"].astype(int).values
X_test = test_df_analysis.drop(columns=["true_label"])

probs = final_model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.30, 0.71, 0.05)  
rows = []

for t in thresholds:
    y_pred = (probs >= t).astype(int)

    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    rows.append({
        "Threshold": round(float(t), 2),
        "F1": f1,
        "Precision": prec,
        "Recall": rec,
        "FP": int(fp),
        "FN": int(fn),
        "TP": int(tp),
        "TN": int(tn),
    })

df = pd.DataFrame(rows)

df_display = df.copy()
for col in ["Precision", "Recall", "F1"]:
    df_display[col] = df_display[col].round(4)

display(df_display)

plt.figure(figsize=(10, 6))
plt.plot(df["Threshold"], df["Precision"], marker="o", label="Precision")
plt.plot(df["Threshold"], df["Recall"], marker="o", label="Recall")
plt.plot(df["Threshold"], df["F1"], marker="o", label="F1")

plt.xlabel("Decision threshold for Stress", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.title("Threshold trade-off in Test set", fontsize=14)
plt.grid(alpha=0.3)
plt.legend()
plt.show()

Adjusting the decision threshold systematically alters the precision–recall balance.

* Lower thresholds (0.30–0.35) produce very high recall (0.94–0.91), meaning most stressed posts are detected. However, this comes at the cost of many false positives (FP = 175 at t=0.30).

* The default threshold (0.50) provides a more balanced profile (Precision = 0.75, Recall = 0.81, F1 = 0.78), reducing false positives substantially while maintaining strong detection performance.

* Higher thresholds (0.60–0.70) increase precision (up to 0.87 at t=0.70), meaning fewer false alarms, but recall drops sharply (down to 0.62), resulting in many missed stress cases (FN = 141 at t=0.70).

F1 peaks around 0.35 (F1 ≈ 0.782) and remains relatively stable between 0.30 and 0.50, indicating that moderate thresholds maintain robust overall performance. Beyond 0.55, declining recall causes F1 to fall.

This confirms the expected trade-off:

* Lower thresholds prioritise case finding (high recall, more workload).

* Higher thresholds prioritise precision (fewer alerts, higher risk of underdetection).

In a mental health screening context, where missing distressed individuals carries asymmetric risk, a threshold slightly below the default (e.g., 0.35–0.40) may be defensible. However, if the model is deployed to triage limited human review capacity, the default threshold around 0.50 represents a pragmatic compromise between detection and operational burden.

Threshold selection should therefore reflect the intended use case rather than default conventions.



### 6.5 Probability Calibration Considerations

While logistic regression produces probabilistic outputs, this study did not explicitly evaluate probability calibration. Calibration assesses whether predicted probabilities correspond to observed outcome frequencies, for example, whether posts predicted at 0.80 stress probability are stressed approximately 80% of the time.

Because threshold selection relies on probability magnitudes, miscalibration could affect operational decisions in real world deployment. Future work may therefore incorporate calibration curves or reliability diagrams to verify probability accuracy, particularly if the model is used for triage or risk scoring.

### 6.6 Error Analysis 

To truly understand model limitations, we analyse both false negatives (missed stress) and false positives (incorrect stress flags), as well as subreddit differences, since they imply different deployment risks.

#### 6.6.1 False Negatives & False Positives

In [ ]:
false_neg = test_df_analysis[(test_df_analysis["true_label"] == 1) & (test_df_analysis["pred_label"] == 0)]
false_pos = test_df_analysis[(test_df_analysis["true_label"] == 0) & (test_df_analysis["pred_label"] == 1)]

cm = confusion_matrix(y_test, test_pred)

plt.figure(figsize=(10,6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="viridis",
    xticklabels=["Non-Stress", "Stress"],
    yticklabels=["Non-Stress", "Stress"]
)

plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Test Set)")
plt.show()

The confusion matrix shows that the model correctly identifies 298 stressed posts and 246 non-stressed posts. It achieves a recall of 0.81 for stress, indicating that most distressed posts are successfully detected. However, 71 stressed posts are missed (false negatives), representing underdetection risk in deployment settings. Additionally, 100 non-stressed posts are flagged as stress (false positives), reflecting overflagging.

The model therefore exhibits recall oriented behaviour: it prioritises identifying stress at the cost of some false alarms. In triage settings, this trade-off may be acceptable if flagged cases are subject to human review.

#### 6.6.2 Most Confident Wrong Predictions 

A granular examination of high confidence failures can reveal the systemic boundaries of language based stress detection. By analyzing "Most Confident" False Negatives (FN) and False Positives (FP), we can identify where computational signals decouple from psychological reality. These errors are not merely random noise but instead, they highlight two primary linguistic challenges:

* Narrative Overpowering: Where a history of positive sentiment masks a current, subtle shift into crisis.
* Affective Intensity Bias: Where high arousal, conflict oriented language is mistaken for active stress, regardless of the user's current state of resolution or reflection.

The following case studies illustrate the tension between lexical frequency and contextual meaning.

In [ ]:
fn_example = false_neg.sort_values("prob_stress").iloc[0]

fp_example = false_pos.sort_values("prob_stress", ascending=False).iloc[0]

print("False Negative Example:")
print("Subreddit:", fn_example["subreddit"])
print("Predicted Probability:", round(fn_example["prob_stress"], 3))
print("True Label:", fn_example["true_label"])
print("Predicted Label:", fn_example["pred_label"])
print("\nText:\n")
print(fn_example["text"])

print("\nFalse Positive Example:")
print("Subreddit:", fp_example["subreddit"])
print("Predicted Probability:", round(fp_example["prob_stress"], 3))
print("True Label:", fp_example["true_label"])
print("Predicted Label:", fp_example["pred_label"])
print("\nText:\n")
print(fp_example["text"])


The False Negative example uses positive or neutral framing: "met our true love," "really happy," "not NOT trying". The only signal of distress is the final sentence: "Things have been rough." As the majority of the post describes a happy timeline, the TF-IDF and LIWC positive emotion scores overwhelm the single "rough" marker. Hence, the model fails to understand the narrative arc and assigns very low stress probability despite contextual distress.

This False Positive post is saturated with high intensity lexical markers: "screaming," "stupid," "moody bitch," "cheating," "abusive asshole". The LIWC negemo and anger scores for this post are likely in the 99th percentile. However, the user is speaking in the past tense ("it took me months afterward," "I remember") and expressing agency/resolution ("it was 100% in my power"). The model cannot distinguish between active trauma (Stress) and processed trauma/reflection (Non-Stress).

These high confidence errors suggest that the model's failures are structural. The False Negative in r/relationships suggests a need for a lower threshold to capture subtle, narrative driven distress. Conversely, the False Positive in r/survivorsofabuse highlights the risk of overtriage in trauma recovery communities, where high intensity language is used for healing rather than expressing active crisis. This reinforces that the model acts as a detector of intensity, not necessarily a diagnostic of clinical state

#### 6.6.3 Error Rates by Subreddit

Error rates vary across communities, indicating domain dependence: some subreddits contain more ambiguous language where stress and non-stress are less separable. This motivates careful validation before transferring the model to new settings (e.g., Singapore specific platforms).

In [ ]:
subreddit_errors = (
    test_df_analysis
    .groupby("subreddit")
    .apply(lambda x: pd.Series({
        "n": len(x),
        "FN_rate": ((x["true_label"] == 1) & (x["pred_label"] == 0)).mean(),
        "FP_rate": ((x["true_label"] == 0) & (x["pred_label"] == 1)).mean(),
    }))
    .sort_values("FN_rate", ascending=False)
)

subreddit_errors

Examination of False Negatives and False Positives reveals systematic patterns, reflecting structural tendencies in how linguistic signals are weighted.

* False Negatives (Stress → Non-Stress):
These errors often occur when distress is implied through context rather than expressed with explicit negative emotion. Many examples use factual or restrained tone, suggesting the model is less sensitive to indirect or “stoic” stress expression. This is a high risk failure mode because stressed users may be missed.

* False Positives (Non-Stress → Stress):
These errors often contain strong emotion words, conflict language, or dramatic phrasing, but do not reflect sustained distress (e.g., resolved situations, venting without impairment). This suggests the model sometimes equates emotional intensity with stress. In deployment, this may increase reviewer workload and raise stigma concerns.


### 6.7 Clinical Ethics and Deployment Risks

#### 6.7.1 Nature of Model and Risks in Deployment

This model detects linguistic patterns statistically associated with stress but it does not diagnose mental health conditions.

The combined analyses reveal:

* Stress prediction is driven primarily by self focus and negative emotion.

* The model performs better for emotionally explicit stress than for subtle or indirect distress.

* Domain variation affects performance.

* Threshold selection materially changes recall–precision balance.

Key risks include:

* False Negatives: Distressed individuals may go undetected.

* False Positives: Users may be unnecessarily flagged.

* Domain Dependence: Contextual differences affect reliability.

* Cultural Variation: Stress expression may differ across populations.

* Privacy Concerns: Large scale linguistic monitoring raises ethical issues.

The model is suitable as a triage or prioritisation tool but should operate within a human-in-the-loop system rather than autonomously.


#### 6.7.2 Ethical Implications of Observed Errors

Confident False Negatives highlight underdetection risk. Subtle or socially embedded distress may not trigger strong negative emotion signals, where individuals express vulnerability within relational contexts. Such errors are clinically significant, as they represent missed opportunities for intervention.


Confident False Positives reveal that emotional intensity can be misinterpreted as stress, creating risk of overflagging.


These findings demonstrate that model errors reflect structural biases in linguistic weighting rather than random noise. Responsible deployment therefore requires:

* Human oversight,
* Context-aware review,
* Transparent threshold calibration,
* Ongoing monitoring for subgroup disparities.

## 7. Conclusion

The final model achieves strong predictive performance (F1 ≈ 0.78), identifying a consistent linguistic signature of stress characterised by heightened self focus and negative emotionality. Its stability on test data suggests reliable internal generalisation within the Dreaddit dataset. However, systematic error patterns reveal structural limitations. Indirect or socially embedded distress is more likely to be missed, while emotionally intense but non-clinical posts are sometimes overflagged. Subreddit variation and threshold sensitivity further demonstrate that stress expression is context dependent rather than uniform.

In the Singapore context, where mental health stigma and cultural norms may encourage restrained or indirect expression of distress, these limitations are particularly salient. Automated detection systems risk underidentifying individuals who communicate distress subtly, while overflagging may create unintended consequences in community or campus environments. Moreover, large scale monitoring raises governance concerns around privacy, consent, and proportionality.

Instead of being deployed as an autonomous diagnostic system, this model should be positioned as a decision support or triage mechanism within structured institutional settings, such as university wellbeing services, moderated online communities, or early intervention programmes. 

Policy deployment would require:

* Threshold calibration prioritising recall in high risk contexts,

* Local validation using Singapore specific linguistic data,

* Mandatory human oversight in escalation decisions,

* Transparent governance frameworks to mitigate surveillance risk.

Ultimately, language based stress detection can augment early identification efforts, but it cannot replace contextual, clinical, or human judgement.